In [1]:
# データセット・ラベルの作成

import split_train_val

ds_org_dir = 'datasets'
ds_root_dir = 'target_datasets'

split_train_val.image_dir_train_val_split(ds_org_dir, ds_root_dir, 0.67)

t_dir = ds_root_dir + '/train'
v_dir = ds_root_dir + '/val'


import generate_label

backup_dir = './model'
label_name = '/label.txt'

labels = generate_label.generate_label(ds_org_dir, backup_dir, label_name)
NUM_CLASSES = len(labels)

datasets/gu copy is done
datasets/choki copy is done
datasets/pa copy is done
processing is done


In [2]:
# インポート

import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

Using plaidml.keras.backend backend.


In [3]:
# ハイパーパラメータ

# 学習率
LEARNING_RATE = 0.001
# エポック
EPOCHS = 20
# バッチサイズ
BATCH_SIZE = 8

In [4]:
# 前処理

IMAGE_SIZE = 64

t_data_gen = ImageDataGenerator(rescale=1./255)
v_data_gen = ImageDataGenerator(rescale=1./255)

t_data = t_data_gen.flow_from_directory(t_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode='rgb', batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)

v_data = v_data_gen.flow_from_directory(v_dir, target_size=(IMAGE_SIZE, IMAGE_SIZE),
    color_mode='rgb', batch_size=BATCH_SIZE, class_mode='categorical', shuffle=True)

Found 111 images belonging to 3 classes.
Found 56 images belonging to 3 classes.


In [7]:
# AIモデルの作成

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES))
model.add(Activation('softmax'))

opt = keras.optimizers.Adam(lr=LEARNING_RATE)

model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_9 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_10 (Activation)   (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 31, 31, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 61504)             0         
__________